In [ ]:
# 这是一个用来得到ec的精细话数据的脚本

# 数据类型
1. vis 
2. uv_10
3. tmax
4. tmin
5. tcc 
6. t2
7. skt
8. lsp(大尺度降水)
9. conv(对流降水)


In [ ]:
class ec_data_point:
    def __init__(self, plot_type,plot_time):
        pass
    def read_data():
        pass
    def render_tojson():
        pass
        
    